In [1]:
import pandas as pd
from pandas import DataFrame

import numpy as np

In [2]:
df = pd.read_csv('./preprocessing-data/preprocessing-data.csv', sep='\t', encoding="UTF-16")

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,id,name,review_average,house_score,score
0,388715,순수원목 A사이드테이블 3colors,4.53,3.888268,5.046576
1,31820,전선정리 멀티탭/박스탭 3colors,4.53,4.180105,5.085348
2,51232,까페 2인 패브릭소파 7colors,4.57,4.429403,5.158468
3,32151,타미 1인 패브릭소파 3colors,4.56,4.499191,5.157740
4,50731,메이블 내츄럴원목 침대 모음전,4.47,4.460260,5.062568


### 상품-score maxtrix

In [4]:
from scipy.sparse import csr_matrix
import sklearn

In [9]:
# 중복되는 데이터 제거
df.drop_duplicates(subset ="name", keep = False, inplace = True)

In [10]:
product_review_avg_house_score_pivot = df.pivot(index = 'name', columns = 'review_average', values = 'house_score').fillna(0)
product_review_avg_house_score_matrix = csr_matrix(product_review_avg_house_score_pivot.values)

In [12]:
product_review_avg_house_score_pivot.head()

review_average,4.42,4.47,4.53,4.56,4.57,4.62,4.65,4.67,4.73,4.88
name,,,,,,,,,,
[쿠폰할인] 스카겐 5칸 서랍장,0.0,0.00000,0.0,0.0,0.000000,4.605484,0.0,0.0,0.000000,0.0
나의 사적인 그림 전시 포스터 / 13 클로드 모네 - 수련 연못 1900 /,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,4.630028,0.0
내추럴 산호 오브제 인테리어 소품 2types,0.0,0.00000,0.0,0.0,0.000000,4.551762,0.0,0.0,0.000000,0.0
메이블 내츄럴원목 침대 모음전,0.0,4.46026,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
뮤즈 2인용 패브릭소파 스툴포함세트,0.0,0.00000,0.0,0.0,4.457464,0.000000,0.0,0.0,0.000000,0.0


In [13]:
product_review_avg_house_score_matrix.shape

(14, 10)

### KNN 적용

In [14]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(product_review_avg_house_score_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
query_index = np.random.choice(product_review_avg_house_score_pivot.shape[0])
print(query_index)
product_review_avg_house_score_pivot.iloc[query_index, :]

13


review_average
4.42    0.000000
4.47    0.000000
4.53    0.000000
4.56    0.000000
4.57    0.000000
4.62    0.000000
4.65    0.000000
4.67    0.000000
4.73    0.000000
4.88    4.653794
Name: 헥스 폴딩 테이블 라지, dtype: float64

In [16]:
query_index = np.random.choice(product_review_avg_house_score_pivot.shape[0])
distances, indices = model_knn.kneighbors(product_review_avg_house_score_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
print(query_index)
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(product_review_avg_house_score_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, product_review_avg_house_score_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

4
Recommendations for 뮤즈 2인용 패브릭소파 스툴포함세트:

1: 타블도트 베이직 14cm 접시 4color 택1, with distance of 0.0:
2: 타미 1인 패브릭소파 3colors, with distance of 1.0:
3: 클라리스 L 플레이트 24cm 4color, with distance of 1.0:
4: 전자레인지 20L 실속형 다이얼식 MEM-GP20W MEM-GP20B 2colors, with distance of 1.0:
5: 에스프레소 스팀 커피머신 2color, with distance of 1.0:


### Collaborative Filtering Using NMF

In [18]:
product_review_avg_house_score_pivot2 = df.pivot(index = 'review_average', columns = 'name', values = 'house_score').fillna(0)
X = pd.DataFrame(product_review_avg_house_score_pivot2.values.T)
X.shape
X.head()

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.00000,0.0,0.0,0.000000,4.605484,0.0,0.0,0.000000,0.0
1,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,4.630028,0.0
2,0.0,0.00000,0.0,0.0,0.000000,4.551762,0.0,0.0,0.000000,0.0
3,0.0,4.46026,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.00000,0.0,0.0,4.457464,0.000000,0.0,0.0,0.000000,0.0


In [19]:
X = pd.DataFrame(product_review_avg_house_score_pivot2.values.T)
X.shape
X.head()

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.00000,0.0,0.0,0.000000,4.605484,0.0,0.0,0.000000,0.0
1,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,4.630028,0.0
2,0.0,0.00000,0.0,0.0,0.000000,4.551762,0.0,0.0,0.000000,0.0
3,0.0,4.46026,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.00000,0.0,0.0,4.457464,0.000000,0.0,0.0,0.000000,0.0


In [20]:
X.columns = product_review_avg_house_score_pivot2.index
X.index = product_review_avg_house_score_pivot2.columns
X.head()

review_average,4.42,4.47,4.53,4.56,4.57,4.62,4.65,4.67,4.73,4.88
name,,,,,,,,,,
[쿠폰할인] 스카겐 5칸 서랍장,0.0,0.00000,0.0,0.0,0.000000,4.605484,0.0,0.0,0.000000,0.0
나의 사적인 그림 전시 포스터 / 13 클로드 모네 - 수련 연못 1900 /,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,4.630028,0.0
내추럴 산호 오브제 인테리어 소품 2types,0.0,0.00000,0.0,0.0,0.000000,4.551762,0.0,0.0,0.000000,0.0
메이블 내츄럴원목 침대 모음전,0.0,4.46026,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
뮤즈 2인용 패브릭소파 스툴포함세트,0.0,0.00000,0.0,0.0,4.457464,0.000000,0.0,0.0,0.000000,0.0


In [21]:
review_count = pd.DataFrame(X[X > 0].count())
len(review_count)

10

In [22]:
from sklearn.decomposition import NMF
model = NMF(n_components=200)

In [23]:
W = model.fit_transform(X)

In [24]:
# H는 Cluster-User matrix
H = model.components_

In [25]:
H.shape

(200, 10)

In [27]:
# H를 dataframe으로 바꾸고 X 컬럼명을 컬럼명으로 세팅
H = pd.DataFrame(np.round(model.components_,2), columns=X.columns)

In [28]:
# W를 dataframe으로 바꾸고 X row명을 row명으로 세팅
W = pd.DataFrame(np.round(model.transform(X),2))
W.index = X.index

In [29]:
H.head()

review_average,4.42,4.47,4.53,4.56,4.57,4.62,4.65,4.67,4.73,4.88
0,0.0,0.0,0.00,0.06,0.00,0.0,0.00,0.0,0.00,0.0
1,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.09,0.0
2,0.0,0.0,0.00,0.00,0.00,0.0,0.58,0.0,0.00,0.0
3,0.0,0.0,0.24,0.00,0.00,0.0,0.00,0.0,0.00,0.0
4,0.0,0.0,0.00,0.00,0.16,0.0,0.00,0.0,0.00,0.0


In [30]:
W.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
name,,,,,,,,,,,,,,,,,,,,,
[쿠폰할인] 스카겐 5칸 서랍장,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,9.18,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
나의 사적인 그림 전시 포스터 / 13 클로드 모네 - 수련 연못 1900 /,0.0,49.21,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
내추럴 산호 오브제 인테리어 소품 2types,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,9.07,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
메이블 내츄럴원목 침대 모음전,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
뮤즈 2인용 패브릭소파 스툴포함세트,0.0,0.00,0.0,0.0,27.68,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# X matrix를 W*H로 reconstruction하여 reconstructed에 저장 colums, index 세팅
reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=X.columns)
reconstructed.index = X.index

In [33]:
reconstructed

review_average,4.42,4.47,4.53,4.56,4.57,4.62,4.65,4.67,4.73,4.88
name,,,,,,,,,,
[쿠폰할인] 스카겐 5칸 서랍장,0.00,0.00,0.00,0.00,0.00,4.59,0.00,0.00,0.00,0.00
나의 사적인 그림 전시 포스터 / 13 클로드 모네 - 수련 연못 1900 /,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.43,0.00
내추럴 산호 오브제 인테리어 소품 2types,0.00,0.00,0.00,0.00,0.00,4.54,0.00,0.00,0.00,0.00
메이블 내츄럴원목 침대 모음전,0.00,4.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
뮤즈 2인용 패브릭소파 스툴포함세트,0.00,0.00,0.00,0.00,4.43,0.00,0.00,0.00,0.00,0.00
시라쿠스 브라운 / 블루 라인 브런치 플레이트 예쁜 접시 4호 (25cm),0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.56,0.00,0.00
에스프레소 스팀 커피머신 2color,0.00,0.00,0.00,4.68,0.00,0.00,0.00,0.00,0.00,0.00
전선정리 멀티탭/박스탭 3colors,0.00,0.00,4.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00
전자레인지 20L 실속형 다이얼식 MEM-GP20W MEM-GP20B 2colors,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.56,0.00,0.00


In [37]:
reconstructed.columns

Float64Index([4.42, 4.47, 4.53, 4.56, 4.57, 4.62, 4.65, 4.67, 4.73, 4.88], dtype='float64', name='review_average')

In [40]:
reconstructed.sort_values(by=[4.88, 4.73, 4.67, 4.65, 4.62, 4.57, 4.56, 4.53],ascending=False)

review_average,4.42,4.47,4.53,4.56,4.57,4.62,4.65,4.67,4.73,4.88
name,,,,,,,,,,
헥스 폴딩 테이블 라지,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.64
나의 사적인 그림 전시 포스터 / 13 클로드 모네 - 수련 연못 1900 /,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.43,0.00
시라쿠스 브라운 / 블루 라인 브런치 플레이트 예쁜 접시 4호 (25cm),0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.56,0.00,0.00
전자레인지 20L 실속형 다이얼식 MEM-GP20W MEM-GP20B 2colors,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.56,0.00,0.00
조안 아쿠아텍스 3중레이어 생활방수 2인용 패브릭소파 3colors,0.00,0.00,0.00,0.00,0.00,0.00,4.58,0.00,0.00,0.00
[쿠폰할인] 스카겐 5칸 서랍장,0.00,0.00,0.00,0.00,0.00,4.59,0.00,0.00,0.00,0.00
내추럴 산호 오브제 인테리어 소품 2types,0.00,0.00,0.00,0.00,0.00,4.54,0.00,0.00,0.00,0.00
타블도트 베이직 14cm 접시 4color 택1,0.00,0.00,0.00,0.00,4.58,0.00,0.00,0.00,0.00,0.00
뮤즈 2인용 패브릭소파 스툴포함세트,0.00,0.00,0.00,0.00,4.43,0.00,0.00,0.00,0.00,0.00
